In [1]:
import tensorflow as tf
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from argparse import ArgumentParser
from configparser import ConfigParser
from keras.models import Sequential
from keras.layers import Activation, Dense, AlphaDropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.utils.generic_utils import get_custom_objects
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import compat
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress Keras/TF warnings
compat.v1.logging.set_verbosity(compat.v1.logging.ERROR) # suppress Keras/TF warnings

In [2]:
# from utils.analysis.tree import Tree
# bkg = Tree(get_qcd_ttbar('dnn'))

In [5]:
tree = Signal('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root')

/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root


In [6]:
id_sort = ak.argsort(tree.jet_signalId, axis=1, ascending=False)[:,:6]
tree.jet_signalId[id_sort]

<Array [[5, 4, 3, 2, 1, 0, ... 3, 2, 1, 0, -1]] type='569341 * var * int32'>

In [76]:
sixb_event = ak.all(tree.jet_signalId[id_sort][:,:6] > -1, axis=1).to_numpy()
n_events = sixb_event.sum()
sixb_event

array([ True, False, False, ..., False,  True, False])

In [77]:
sorted_id = tree.jet_signalId[id_sort][:,:6].to_numpy()

# turn jet id 0 into higgs id 1 (jet id 1 is already higgs id 1 - HX)
sorted_id = np.where(sorted_id == 0, 1, sorted_id)
# turn jet id 3 into higgs id 2 (jet id 2 is already higgs id 2 - H1)
sorted_id = np.where(sorted_id == 3, 2, sorted_id)
# turn jet id 4 and 5 into higgs id 3 (H2)
sorted_id = np.where(sorted_id == 4, 3, sorted_id)
sorted_id = np.where(sorted_id == 5, 3, sorted_id)
# turn jet id -1 into higgs id 0 (not a Higgs)
sorted_id = np.where(sorted_id == -1, 0, sorted_id)

# n_events = sorted_id[sixb_event].shape[0]


sixb_sorted_id = sorted_id[sixb_event][:n_events]
xxxx_sorted_id = sorted_id[~sixb_event][:n_events]

In [78]:
sorted_id.shape, sorted_id[sixb_event].shape, sorted_id[~sixb_event].shape

((569341, 6), (176884, 6), (392457, 6))

In [79]:
sorted_id.shape, sixb_sorted_id.shape, xxxx_sorted_id.shape

((569341, 6), (176884, 6), (176884, 6))

In [80]:
sixb_sorted_id, xxxx_sorted_id

(array([[3, 3, 2, 2, 1, 1],
        [3, 3, 2, 2, 1, 1],
        [3, 3, 2, 2, 1, 1],
        ...,
        [3, 3, 2, 2, 1, 1],
        [3, 3, 2, 2, 1, 1],
        [3, 3, 2, 2, 1, 1]], dtype=int32),
 array([[3, 2, 2, 1, 1, 0],
        [3, 2, 2, 1, 1, 0],
        [3, 3, 2, 1, 0, 0],
        ...,
        [3, 3, 2, 1, 1, 0],
        [3, 3, 2, 1, 0, 0],
        [3, 2, 2, 1, 1, 0]], dtype=int32))

In [81]:
from itertools import combinations

combos = list(combinations(np.arange(6),2))
combos

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5)]

In [82]:
sixb_particles = []
for i in range(6):
    sixb_particles.append(Particle(kin_dict={
        'pt' : tree.jet_ptRegressed[id_sort][sixb_event][:,i],
        'eta' : tree.jet_eta[id_sort][sixb_event][:,i],
        'phi' : tree.jet_phi[id_sort][sixb_event][:,i],
        'm' : tree.jet_mRegressed[id_sort][sixb_event][:,i]
        }))
sixb_particles

xxxx_particles = []
for i in range(6):
    xxxx_particles.append(Particle(kin_dict={
        'pt' : tree.jet_ptRegressed[id_sort][~sixb_event][:,i],
        'eta' : tree.jet_eta[id_sort][~sixb_event][:,i],
        'phi' : tree.jet_phi[id_sort][~sixb_event][:,i],
        'm' : tree.jet_mRegressed[id_sort][~sixb_event][:,i]
        }))

In [83]:
sixb_pt = np.column_stack([particle.pt.to_numpy() for particle in sixb_particles])
sixb_eta = np.column_stack([particle.eta.to_numpy() for particle in sixb_particles])

xxxx_pt = np.column_stack([particle.pt.to_numpy() for particle in xxxx_particles])
xxxx_eta = np.column_stack([particle.eta.to_numpy() for particle in xxxx_particles])

sixb_deltaPhi = []
sixb_deltaEta = []
sixb_deltaR = []

xxxx_deltaPhi = []
xxxx_deltaEta = []
xxxx_deltaR = []

# boost_dPhi = []
# boost_dEta = []
# boost_dR = []
# boost_m = []

for combo in combos:
    i,j = combo[0], combo[1]

    part_1 = sixb_particles[i]
    part_2 = sixb_particles[j]

    sixb_deltaPhi.append(part_1.deltaPhi(part_2).to_numpy())
    sixb_deltaEta.append(part_1.deltaEta(part_2).to_numpy())
    sixb_deltaR.append(part_1.deltaR(part_2).to_numpy())

    part_1 = xxxx_particles[i]
    part_2 = xxxx_particles[j]

    xxxx_deltaPhi.append(part_1.deltaPhi(part_2).to_numpy())
    xxxx_deltaEta.append(part_1.deltaEta(part_2).to_numpy())
    xxxx_deltaR.append(part_1.deltaR(part_2).to_numpy())

sixb_deltaPhi = np.column_stack((sixb_deltaPhi))
sixb_deltaEta = np.column_stack((sixb_deltaEta))
sixb_deltaR = np.column_stack((sixb_deltaR))

xxxx_deltaPhi = np.column_stack((xxxx_deltaPhi))
xxxx_deltaEta = np.column_stack((xxxx_deltaEta))
xxxx_deltaR = np.column_stack((xxxx_deltaR))

In [130]:
# sixb_features = np.column_stack((sixb_pt, sixb_eta, sixb_deltaPhi, sixb_deltaEta, sixb_deltaR))
# xxxx_features = np.column_stack((xxxx_pt, xxxx_eta, xxxx_deltaPhi, xxxx_deltaEta, xxxx_deltaR))
# sixb_features = np.column_stack((sixb_pt, sixb_eta, sixb_deltaPhi, sixb_deltaR))
# xxxx_features = np.column_stack((xxxx_pt, xxxx_eta, xxxx_deltaPhi, xxxx_deltaR))
sixb_features = np.column_stack((sixb_pt, sixb_eta, sixb_deltaPhi))
xxxx_features = np.column_stack((xxxx_pt, xxxx_eta, xxxx_deltaPhi))

features = np.row_stack((sixb_features[:n_events], xxxx_features[:n_events]))
labels = np.row_stack((sixb_sorted_id, xxxx_sorted_id))

n_inputs = features.shape[1]
n_outputs = labels.shape[1]

print(features.shape, labels.shape)
print(n_inputs, n_outputs)

(353768, 27) (353768, 6)
27 6


In [179]:
# Hidden hyperparameters
hidden_activations = 'selu'
nodes              = [20,40,20]
nlayers            = len(nodes)

# Output hyperparameters
output_activation  = 'linear'
output_nodes       = n_outputs

optimizer_name     = 'nadam'
loss_function      = 'mae'
nepochs            = 50
batch_size         = 100

In [180]:
scaler = MinMaxScaler()
scaler.fit(features)
x = scaler.transform(features)

val_size = 0.10
x_train, x_val, y_train, y_val = train_test_split(x, labels, test_size=val_size)

param_dim = n_inputs

In [181]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

In [182]:
model = Sequential()

model.add(Dense(nodes[0], input_dim=param_dim, activation=hidden_activations, kernel_initializer='he_uniform'))
for i in range(1,nlayers):
    # model.add(Dense(int(nodes[i]), activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    model.add(Dense(int(nodes[i]), activation=hidden_activations, kernel_initializer='he_uniform'))
    # model.add(Dense(int(nodes[i]), kernel_initializer='lecun_normal', activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    # if bool(args.dropout): model.add(AlphaDropout(0.2)) 
model.add(Dense(output_nodes, activation=output_activation))

In [183]:
es = EarlyStopping(monitor='loss', restore_best_weights=True, patience=10)

In [184]:
met = ['accuracy']

In [185]:
lr = 0.0001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-7

In [186]:
# optimizer = 'nadam'
optimizer = tf.keras.optimizers.Nadam(
    learning_rate=lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, name="nadam"
)

In [187]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, name="adam"
)

In [188]:
model.compile(loss=loss_function, 
              optimizer=optimizer, 
              metrics=met)

In [189]:
nn_info_list = [
    f"Input parameters:            {param_dim}\n",
    f"Optimizer:                   {optimizer}\n",
    f"Learning Rate:               {lr}\n",
    f"beta_1:                      {beta1}\n",
    f"beta_2:                      {beta2}\n",
    f"epsilon:                     {epsilon}\n",
    f"Loss:                        {loss_function}\n",
    f"Num epochs:                  {nepochs}\n",
    f"Batch size:                  {batch_size}\n",
    f"Num hidden layers:           {nlayers}\n",
    f"Input activation function:   {hidden_activations}\n",
    f"Hidden layer nodes:          {nodes}\n",
    f"Hidden activation functions: {hidden_activations}\n",
    f"Num output nodes:            {output_nodes}\n",
    f"Output activation function:  {output_activation.capitalize()}\n"]

for line in nn_info_list:
    print(line[:-1])

Input parameters:            27
Optimizer:                   <keras.optimizers.optimizer_v2.adam.Adam object at 0x7f19ec165580>
Learning Rate:               0.0001
beta_1:                      0.9
beta_2:                      0.999
epsilon:                     1e-07
Loss:                        mae
Num epochs:                  50
Batch size:                  100
Num hidden layers:           3
Input activation function:   selu
Hidden layer nodes:          [20, 40, 20]
Hidden activation functions: selu
Num output nodes:            6
Output activation function:  Linear


In [190]:
# tf.config.run_functions_eagerly(True)

In [191]:
history = model.fit(x_train,
                    y_train, 
                    validation_data=(x_val, y_val), 
                    epochs=nepochs, 
                    batch_size=batch_size, 
                    callbacks=[es])

model.summary()

Epoch 1/50
3184/3184 [==============================] - 11s 3ms/step - loss: 0.3857 - accuracy: 0.6089 - val_loss: 0.2554 - val_accuracy: 0.6699
Epoch 2/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.2325 - accuracy: 0.6702 - val_loss: 0.2057 - val_accuracy: 0.7415
Epoch 3/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.1930 - accuracy: 0.7548 - val_loss: 0.1917 - val_accuracy: 0.8221
Epoch 4/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.1867 - accuracy: 0.7721 - val_loss: 0.1875 - val_accuracy: 0.7882
Epoch 5/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.1835 - accuracy: 0.7725 - val_loss: 0.1847 - val_accuracy: 0.7803
Epoch 6/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.1809 - accuracy: 0.7710 - val_loss: 0.1823 - val_accuracy: 0.7568
Epoch 7/50
3184/3184 [==============================] - 12s 4ms/step - loss: 0.1781 - accuracy: 0.7695 - val_loss: 0.1790 - val_ac

In [192]:
test_tree = Signal('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400/ntuple.root')

/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400/ntuple.root


In [194]:
test_particles = []
for i in range(6):
    test_particles.append(Particle(kin_dict={
        'pt' : test_tree.jet_ptRegressed[:,i],
        'eta' : test_tree.jet_eta[:,i],
        'phi' : test_tree.jet_phi[:,i],
        'm' : test_tree.jet_mRegressed[:,i]
        }))
test_particles

In [195]:
test_pt = np.column_stack([particle.pt.to_numpy() for particle in test_particles])
test_eta = np.column_stack([particle.eta.to_numpy() for particle in test_particles])

test_deltaPhi = []
test_deltaEta = []
test_deltaR = []

for combo in combos:
    i,j = combo[0], combo[1]

    part_1 = test_particles[i]
    part_2 = test_particles[j]

    test_deltaPhi.append(part_1.deltaPhi(part_2).to_numpy())
    test_deltaEta.append(part_1.deltaEta(part_2).to_numpy())
    test_deltaR.append(part_1.deltaR(part_2).to_numpy())

test_deltaPhi = np.column_stack((test_deltaPhi))
test_deltaEta = np.column_stack((test_deltaEta))
test_deltaR = np.column_stack((test_deltaR))

In [196]:
test_features = np.column_stack((test_pt, test_eta, test_deltaPhi))

In [197]:
test_id = test_tree.jet_signalId[:,:6].to_numpy()

test_id = np.where(test_id == 0, 1, test_id)
test_id = np.where(test_id == 3, 2, test_id)
test_id = np.where(test_id == 4, 3, test_id)
test_id = np.where(test_id == 5, 3, test_id)
test_id = np.where(test_id == -1, 0, test_id)

In [199]:
test_features.shape, test_id.shape

((55952, 27), (55952, 6))

In [213]:
test_input = scaler.transform(test_features)

In [214]:
test_output = model.predict(test_input)
test_output.shape

1749/1749 [==============================] - 3s 2ms/step


(55952, 6)

In [232]:
model.predict(test_input)

1749/1749 [==============================] - 3s 2ms/step


array([[ 3.0009506e+00,  2.0660005e+00,  2.0005045e+00,  9.9621248e-01,
         1.0008233e+00,  3.5982803e-03],
       [ 2.9948192e+00,  1.9261633e+00,  1.9944764e+00,  1.0457910e+00,
         9.6157491e-01,  6.6370316e-02],
       [ 3.0022426e+00,  1.9630197e+00,  1.9998080e+00,  9.7861636e-01,
         1.0003309e+00,  4.4650584e-04],
       ...,
       [ 2.9977434e+00,  2.0169146e+00,  1.9999269e+00,  9.7949362e-01,
         1.0011699e+00, -1.9192152e-02],
       [ 3.0006115e+00,  2.0231431e+00,  2.0019224e+00,  9.7862887e-01,
         9.8991638e-01, -1.9088797e-02],
       [ 3.0017829e+00,  2.0353847e+00,  1.9995835e+00,  9.8529506e-01,
         9.9525744e-01,  3.8823262e-03]], dtype=float32)

In [215]:
test_output = np.round(test_output).astype(int)
test_output

array([[3, 2, 2, 1, 1, 0],
       [3, 2, 2, 1, 1, 0],
       [3, 2, 2, 1, 1, 0],
       ...,
       [3, 2, 2, 1, 1, 0],
       [3, 2, 2, 1, 1, 0],
       [3, 2, 2, 1, 1, 0]])

In [216]:
test_id

array([[1, 3, 2, 2, 3, 1],
       [3, 1, 0, 1, 2, 0],
       [2, 3, 2, 1, 1, 0],
       ...,
       [3, 2, 1, 1, 2, 0],
       [2, 2, 1, 1, 3, 0],
       [1, 1, 2, 3, 2, 0]], dtype=int32)

In [1]:
np.any(test_id == 0, axis=1)

NameError: name 'test_id' is not defined

In [217]:
test_output.shape, test_id.shape

((55952, 6), (55952, 6))

In [219]:
test_truth_values = np.equal(test_output, test_id)

In [220]:
test_truth_values.sum() / np.ones_like(test_truth_values).sum()

0.30509186445524733

In [223]:
np.average(test_truth_values.sum(axis=1))

1.830551186731484

In [230]:
test_truth_values[test_id > 0].sum() / (test_id > 0).sum()

0.2597955894204704